# How to insert, update, and delete a single record

To perform insert, update, and delete operations for a single record on a database using C# and Dapper, you can use the <code>Execute</code> or <code>ExecuteAsync</code> methods of Dapper to run SQL statements or stored procedures with parameters. These methods return the number of affected rows by the operation. You can also use the <code>Query</code> or <code>QueryAsync</code> methods of Dapper to get the values from the inserted, updated, or deleted rows, by using clauses such as OUTPUT in your SQL statements. For example:



In [7]:
// Connection string
#load "AppSettings.cs"

In [8]:
#r "nuget:Microsoft.Data.SqlClient"
#r "nuget:System.ComponentModel"
#r "nuget:Dapper"

Installed Packages Dapper, 2.1.24 Microsoft.Data.SqlClient, 5.1.2 System.ComponentModel, 4.3.0

In [9]:
using Microsoft.Data.SqlClient;
using System;
using System.Data;
using Dapper;

public class Product
{    
    public int ProductID { get; set; }
    public string Name { get; set; }
    public string ProductNumber { get; set; }
    public string Color { get; set; }
    public decimal StandardCost { get; set; }
    public decimal ListPrice { get; set; }
    public int SafetyStockLevel { get; set; }
    public int ReorderPoint { get; set; }
    public int DaysToManufacture { get; set; }
    public DateTime SellStartDate { get; set; }
}

To insert a new record into the Person.Person table, you can use the following code:

In [10]:
var rand = new Random();

var product = new Product() { Name = "Bike Helmet-" + rand.Next(1000, 9999), ProductNumber = "BK-H01-" + rand.Next(1000, 9999), Color = "Red", 
                              StandardCost = 15.50m, ListPrice = 34.99m, SafetyStockLevel = 999, ReorderPoint = 111, DaysToManufacture = 1, 
                              SellStartDate = new DateTime(2000, 1, 2) };

int productID = 0;
using (var connection = new SqlConnection(connectionString))
{
    // Build the SQL query with parameters
    var sql = $@"INSERT INTO Production.Product (Name, ProductNumber, Color, StandardCost, ListPrice, SafetyStockLevel, ReorderPoint, 
                 DaysToManufacture, SellStartDate) OUTPUT INSERTED.ProductID 
                 VALUES (@Name, @ProductNumber, @Color, @StandardCost, @ListPrice, @SafetyStockLevel, @ReorderPoint, @DaysToManufacture, @SellStartDate) ";

    // Execute the query and get the id
    var ids = await connection.QueryAsync<int>(sql, product);

    // Print the ids
    productID = ids.FirstOrDefault<int>();
    Console.WriteLine(productID);
}

1098


To update the above record:

In [11]:
var product = new Product() { ProductID = productID, Name = "Bearing Ball-" + rand.Next(1000, 9999), ProductNumber = "BA-" + rand.Next(1000, 9999), Color = "Black", 
                              StandardCost = 0.50m, ListPrice = 0.99m, SafetyStockLevel = 555, ReorderPoint = 747, DaysToManufacture = 0, SellStartDate = DateTime.Now };

using (var connection = new SqlConnection(connectionString))
{
    // Build the SQL query with parameters
    var sql = $@"UPDATE Production.Product SET Name = @Name, ProductNumber = @ProductNumber, Color = @Color, StandardCost = @StandardCost, 
                 ListPrice = @ListPrice, SafetyStockLevel = @SafetyStockLevel, ReorderPoint = @ReorderPoint, DaysToManufacture = @DaysToManufacture, 
                 SellStartDate = @SellStartDate WHERE ProductID = @ProductID";

    // Execute the query and get the number of affected rows
    var rows = await connection.ExecuteAsync(sql, product);

    // Print the rows
    Console.WriteLine(rows);
}

1


To delete the above record:

In [12]:
using (var connection = new SqlConnection(connectionString))
{
    // Build the SQL query with parameter
    var sql = "DELETE FROM Production.Product WHERE ProductID = @ProductID";

    // Execute the query and get the number of affected rows
    var rows = await connection.ExecuteAsync(sql, new { ProductID = productID });

    // Print the rows
    Console.WriteLine(rows);
}

1
